In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import sqlalchemy as sa

import database as db

from images import get_list_of_images_invalid, separate_and_copy_images
from models import County, DataTrustedIdentifier

## Database connection

In [3]:
engine, session = db.connect()
engine.echo = False

2023-03-28 09:49:26,615 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-28 09:49:26,616 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-28 09:49:26,617 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-28 09:49:26,617 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-28 09:49:26,619 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-28 09:49:26,619 INFO sqlalchemy.engine.Engine [raw sql] {}


## List of images invalid

In [4]:
list_images_invalid = get_list_of_images_invalid()
pd.DataFrame(list_images_invalid).head(3)

,barcode,reason
0,INPA0248526,horizontal
1,INPA0248523,horizontal
2,INPA0248528,horizontal


## BR dataset

In [5]:
query = session.query(County).distinct().all()

uf_unaccented_lower, state_unaccented_lower, county_unaccented_lower = db.get_list_uf_state_county(query)

## Define variable for build dataset

In [6]:
list_minimum_image=[5]#, 10, 20]
list_color=['rgb']#, 'grayscale']
list_image_size=['256']#, '400', '512']
path_out = '/home/xandao/Imagens/br_dataset'

## copy images

In [15]:
level = DataTrustedIdentifier.specific_epithet_trusted
if level == DataTrustedIdentifier.genus_trusted:
    condition_level = DataTrustedIdentifier.genus_trusted.is_not(None)
else:
    condition_level = DataTrustedIdentifier.specific_epithet_trusted.is_not(None)

condition = sa.and_(DataTrustedIdentifier.country_trusted =='Brasil',
                condition_level,
                 sa.or_(uf_unaccented_lower, state_unaccented_lower))

separate_and_copy_images(condition, level, list_color, list_images_invalid, list_minimum_image, list_image_size, path_out, session)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



8
[['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/MBM143148.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/MBM143137.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/MBM045087.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/MBM045081.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/R010054930_001.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mask/NY00559081_01.jpeg'], ['/media/xandao/c2f58d30-ff2c-47f7-95af-91ad6fd69760/dataset/dataset-52k-sp-RGB-resize-xcf-mask-segmented/RGB/256/w_pred_mas

SystemExit: 

## Close connection

In [43]:
session.close()
engine.dispose()